In [1]:
import datetime as dt
from utility import (
    read_sql,
    fd_alive_funds,
    fd_basicinfo,
    fd_typeclass,
    fd_hshkiport,
    fd_assetportfolio,
    fd_derieden,
    nearest_report_date
)
import pandas as pd
from PyFin.api import makeSchedule
from PyFin.api import BizDayConventions

In [2]:
# 获取回溯的报告日

today = dt.datetime.today()
date_3yrs_ago = today.replace(year=today.year - 3)
report_dates_begin = nearest_report_date(date_3yrs_ago.strftime("%Y%m%d"))
current_date = today.strftime("%Y%m%d")

In [5]:
# 获取所有目标债券型基金（清算结束日期晚于当前日）

sec_codes = fd_alive_funds(report_dates_begin, 2, 203)

In [6]:
sec_codes

['1030000011',
 '1030000012',
 '1030000014',
 '1030000015',
 '1030000022',
 '1030000031',
 '1030000042',
 '1030000043',
 '1030000049',
 '1030000054',
 '1030000059',
 '1030000063',
 '1030000074',
 '1030000080',
 '1030000084',
 '1030000086',
 '1030000101',
 '1030000102',
 '1030000108',
 '1030000109',
 '1030000112',
 '1030000118',
 '1030000124',
 '1030000126',
 '1030000131',
 '1030000133',
 '1030000135',
 '1030000141',
 '1030000173',
 '1030000180',
 '1030000195',
 '1030000198',
 '1030000201',
 '1030000205',
 '1030000234',
 '1030000238',
 '1030000241',
 '1030000253',
 '1030000279',
 '1030000291',
 '1030000292',
 '1030000295',
 '1030000296',
 '1030000303',
 '1030000311',
 '1030000312',
 '1030000314',
 '1030000316',
 '1030000318',
 '1030000354',
 '1030000360',
 '1030000361',
 '1030000364',
 '1030000365',
 '1030000380',
 '1030000391',
 '1030000495',
 '1030000496',
 '1030000499',
 '1030000500',
 '1030000502',
 '1030000503',
 '1030000505',
 '1030000511',
 '1030000512',
 '1030000515',
 '10300005

In [4]:
# 获取相关组合情况

asset_port = fd_assetportfolio(security_ids, report_dates_begin, current_date).fillna(0)
asset_port["CONVBDRTO_TO_BDRTO"] = asset_port["CONVBDRTO"] / asset_port["BDRTO"] * 100

In [5]:
last_portfolio = asset_port.groupby("SECURITYID").last()[["BDRTO"]]
last_4_cov_portfolio = asset_port.groupby("SECURITYID").rolling(window=4).mean().groupby(level=0).last()[["CONVBDRTO_TO_BDRTO"]]
last_4_stk_portfolio = asset_port.groupby("SECURITYID").rolling(window=4).mean().groupby(level=0).last()[["EQUITYINVERTO"]]

In [6]:
agg_port = pd.concat([last_portfolio, last_4_cov_portfolio, last_4_stk_portfolio], axis=1).reset_index()
agg_port = pd.merge(agg_port, fd_typeclass(agg_port.SECURITYID.tolist(), current_date))
basic_info = fd_basicinfo(agg_port.SECURITYID.tolist())
agg_port = pd.merge(agg_port, basic_info)

In [7]:
agg_port.L3NAME.unique()

array(['普通债券型基金(一级A类)', '普通债券型基金(一级B/C类)', '普通债券型基金(二级A类)',
       '普通债券型基金(二级B/C类)', '中短期标准债券型基金', '普通债券型基金(可投转债A类)',
       '普通债券型基金(可投转债B类)', '指数债券型基金(A类)', '长期标准债券型基金(A类)', '可转换债券型基金(A类)',
       '可转换债券型基金(B/C类)', '债券型分级子基金(优先份额)', '债券型分级子基金(进取份额)',
       '指数债券型基金(B/C类)', '长期标准债券型基金(B/C类)', '短期理财债券型基金(A类)',
       '短期理财债券型基金(B/C类)'], dtype=object)

In [8]:
# 分类
agg_port.loc[(agg_port["BDRTO"] >= 80) & (agg_port.L2NAME == "短期理财债券型基金"), "债券型（子类）"] = "短期纯债型基金"
agg_port.loc[(agg_port["BDRTO"] >= 80) & (agg_port["CONVBDRTO_TO_BDRTO"] >= 80), "债券型（子类）"] = "可转债基金"
agg_port.loc[(agg_port["BDRTO"] >= 80) & (agg_port["CONVBDRTO_TO_BDRTO"] >= 5) & (agg_port["CONVBDRTO_TO_BDRTO"] < 80) & (agg_port["EQUITYINVERTO"] <= 0.1), "债券型（子类）"] = "可投转债型债券基金"
agg_port.loc[(agg_port["BDRTO"] >= 80) & (agg_port["EQUITYINVERTO"] >= 0.1) & (~agg_port["债券型（子类）"].isin(["可投转债型债券基金", "可转债基金"])), "债券型（子类）"] = "可投股票型债券基金"
agg_port.loc[(agg_port["BDRTO"] >= 80) & (agg_port["L2NAME"] == "指数债券型基金") & (~agg_port["债券型（子类）"].isin(["可投转债型债券基金", "可转债基金"])), "债券型（子类）"] = "被动指数型债券基金"
agg_port.loc[pd.isnull(agg_port["债券型（子类）"]), "债券型（子类）"] = "中长期纯债型基金"

In [9]:
agg_port.groupby("债券型（子类）").count()

,SECURITYID,BDRTO,CONVBDRTO_TO_BDRTO,EQUITYINVERTO,L1CODE,L1NAME,L2CODE,L2NAME,L3CODE,L3NAME,FDNAME,SNAMECOMP,FSYMBOL,FDNATURE,INVESTSTYLE
债券型（子类）,,,,,,,,,,,,,,,
中长期纯债型基金,638,638,625,632,638,638,638,638,638,638,638,450,638,638,638
可投股票型债券基金,370,370,370,370,370,370,370,370,370,370,370,321,370,370,370
可投转债型债券基金,87,87,87,87,87,87,87,87,87,87,87,77,87,87,87
可转债基金,41,41,41,41,41,41,41,41,41,41,41,36,41,41,41
短期纯债型基金,55,55,54,55,55,55,55,55,55,55,55,54,55,55,55
被动指数型债券基金,17,17,17,17,17,17,17,17,17,17,17,11,17,17,17


## 1.1 Put it all totether

In [10]:
def create_bond_fund_info(trade_dt):
    trade_dt = dt.datetime.strptime(trade_dt, "%Y%m%d")
    date_3yrs_ago = trade_dt.replace(year=trade_dt.year - 3)
    report_dates_begin = nearest_report_date(date_3yrs_ago.strftime("%Y%m%d"))
    current_date = trade_dt.strftime("%Y%m%d")
    
    security_ids = fd_alive_funds(report_dates_begin, 3)
    basic_info = fd_basicinfo(security_ids, current_date)
    type_info = fd_typeclass(security_ids, current_date)
    asset_port = fd_assetportfolio(security_ids, report_dates_begin, current_date).fillna(0)
    asset_port["CONVBDRTO_TO_BDRTO"] = asset_port["CONVBDRTO"] / asset_port["BDRTO"] * 100
    last_portfolio = asset_port.groupby("SECURITYID").last()[["BDRTO"]]
    last_4_cov_portfolio = asset_port.groupby("SECURITYID").rolling(window=4).mean().groupby(level=0).last()[["CONVBDRTO_TO_BDRTO"]]
    last_4_stk_portfolio = asset_port.groupby("SECURITYID").rolling(window=4).mean().groupby(level=0).last()[["EQUITYINVERTO"]]

    df = pd.merge(basic_info, type_info, on="SECURITYID")
    df = pd.merge(df, last_portfolio, on="SECURITYID", how="left").drop_duplicates()
    df = pd.merge(df, last_4_stk_portfolio, on="SECURITYID", how="left").drop_duplicates()
    df = pd.merge(df, last_4_cov_portfolio, on="SECURITYID", how="left").drop_duplicates()
    
    df.loc[(df["BDRTO"] >= 80) & (df.L2NAME == "短期理财债券型基金"), "债券型（子类）"] = "短期纯债型基金"
    df.loc[(df["BDRTO"] >= 80) & (df["CONVBDRTO_TO_BDRTO"] >= 80), "债券型（子类）"] = "可转债基金"
    df.loc[(df["BDRTO"] >= 80) & (df["CONVBDRTO_TO_BDRTO"] >= 5) & (df["CONVBDRTO_TO_BDRTO"] < 80) & (df["EQUITYINVERTO"] <= 0.1), "债券型（子类）"] = "可投转债型债券基金"
    df.loc[(df["BDRTO"] >= 80) & (df["EQUITYINVERTO"] >= 0.1) & (~df["债券型（子类）"].isin(["可投转债型债券基金", "可转债基金"])), "债券型（子类）"] = "可投股票型债券基金"
    df.loc[(df["BDRTO"] >= 80) & (df["L2NAME"] == "指数债券型基金") & (~df["债券型（子类）"].isin(["可投转债型债券基金", "可转债基金"])), "债券型（子类）"] = "被动指数型债券基金"
    df.loc[pd.isnull(df["债券型（子类）"]), "债券型（子类）"] = "中长期纯债型基金"
    
    return df

In [11]:
%%time

df = create_bond_fund_info("20211111")#.groupby( "债券型（子类）")

Wall time: 5.99 s


In [18]:
%%time

create_bond_fund_info("20211111").groupby( "债券型（子类）").count()

Wall time: 2.66 s


In [20]:
ddf = create_bond_fund_info("20211111")

In [21]:
ddf

,SECURITYID,FDNAME,SNAMECOMP,FSYMBOL,FDNATURE,INVESTSTYLE,L1CODE,L1NAME,L2CODE,L2NAME,L3CODE,L3NAME,BDRTO,EQUITYINVERTO,CONVBDRTO_TO_BDRTO,债券型（子类）
0,1030000011,华夏债券投资基金,华夏债券,001001,证券投资基金,收益型,3,债券基金,3.2,普通债券型基金,3.2.1,普通债券型基金(一级A类),110.27,6.195000e+00,3.610785e+01,可投股票型债券基金
1,1030000012,华夏债券投资基金,华夏债券,001003,证券投资基金,收益型,3,债券基金,3.2,普通债券型基金,3.2.2,普通债券型基金(一级B/C类),90.79,6.250000e+00,3.012725e+01,可投股票型债券基金
2,1030000014,华夏希望债券型证券投资基金,华夏希望债券,001011,证券投资基金,收益型,3,债券基金,3.2,普通债券型基金,3.2.3,普通债券型基金(二级A类),87.45,6.267500e+00,2.437650e+00,可投股票型债券基金
3,1030000015,华夏希望债券型证券投资基金,华夏希望债券,001013,证券投资基金,收益型,3,债券基金,3.2,普通债券型基金,3.2.4,普通债券型基金(二级B/C类),84.83,1.151000e+01,1.630190e+00,可投股票型债券基金
4,1030000022,国泰金龙债券证券投资基金,国泰金龙债券,020002,证券投资基金,收益型,3,债券基金,3.2,普通债券型基金,3.2.1,普通债券型基金(一级A类),105.18,0.000000e+00,1.218145e+01,可投转债型债券基金
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1220,1030002083,交银施罗德理财60天债券型证券投资基金,交银理财60天债券,519722,证券投资基金,收益型,3,债券基金,3.6,短期理财债券型基金,3.6.2,短期理财债券型基金(B/C类),85.64,1.776357e-15,7.105427e-15,短期纯债型基金
1221,1030001814,易方达月月利理财债券型证券投资基金,易方达月月利理财债券,110050,证券投资基金,稳健成长型,3,债券基金,3.6,短期理财债券型基金,3.6.1,短期理财债券型基金(A类),40.81,2.664535e-15,7.105427e-15,中长期纯债型基金
1222,1030001815,易方达月月利理财债券型证券投资基金,易方达月月利理财债券,110051,证券投资基金,稳健成长型,3,债券基金,3.6,短期理财债券型基金,3.6.2,短期理财债券型基金(B/C类),49.96,2.664535e-15,7.105427e-15,中长期纯债型基金
1223,1030010003,易方达掌柜季季盈理财债券型证券投资基金,易方达掌柜季季盈理财债券,000833,证券投资基金,收益型,3,债券基金,3.6,短期理财债券型基金,3.6.1,短期理财债券型基金(A类),49.46,3.552714e-15,2.842171e-14,中长期纯债型基金


# 2. 基金指数
--------------------

In [13]:
# 我们只计算指数的每日收益
# 在确定指数的基准日之后，可以直接使用收益计算指数的值。
# 使用 TQ_FD_DERIVEDN 获取基金的净值情况

start_dt = "2015-01-31"
final_trade_dt = "2021-11-09"
nav_type = "REPAIRUNITNAV"

rebalance_dates = [d.strftime("%Y%m%d") for d in makeSchedule(start_dt, final_trade_dt, tenor="3M", calendar="china.sse", dateRule=BizDayConventions.ModifiedFollowing)]

In [14]:
dfs = []

for i, trade_dt in enumerate(rebalance_dates[1:]):
    pre_trade_dt = rebalance_dates[i]
    print(pre_trade_dt, trade_dt)
    fund_info = create_bond_fund_info(trade_dt)
    pre_nav_info = fd_derieden(fund_info.SECURITYID.tolist(), pre_trade_dt).rename(columns={nav_type: "PRE" + nav_type})
    nav_info = fd_derieden(fund_info.SECURITYID.tolist(), trade_dt)

    total_df = pd.merge(fund_info, nav_info, on=["SECURITYID"])
    total_df = pd.merge(total_df, pre_nav_info, on=["SECURITYID"])
    total_df["chg."] = total_df[nav_type] / total_df["PRE" + nav_type] - 1.0
    res = total_df.groupby("债券型（子类）")[["chg."]].mean()
    dfs.append(res)

20150130 20150430
20150430 20150731
20150731 20151030
20151030 20160129
20160129 20160429
20160429 20160729
20160729 20161031
20161031 20170126
20170126 20170428
20170428 20170731
20170731 20171031
20171031 20180131
20180131 20180427
20180427 20180731
20180731 20181031
20181031 20190131
20190131 20190430
20190430 20190731
20190731 20191031
20191031 20200123
20200123 20200430
20200430 20200731
20200731 20201030
20201030 20210129
20210129 20210430
20210430 20210730
20210730 20211029
20211029 20211109


In [15]:
final_report = pd.concat(dfs, keys=rebalance_dates[1:]).reset_index()
final_report.pivot_table(index="level_0", columns="债券型（子类）", values="chg.").to_excel("030_债券型基金_bak.xlsx")

In [16]:
final_report.pivot_table(index="level_0", columns="债券型（子类）", values="chg.")

债券型（子类）,中长期纯债型基金,可投股票型债券基金,可投转债型债券基金,可转债基金,短期纯债型基金,被动指数型债券基金
level_0,,,,,,
20150430,0.035524,0.073317,0.046774,0.181991,NaN,0.018172
20150731,0.015343,0.003398,0.009793,-0.181767,NaN,0.001508
20151030,0.019825,0.017772,0.022953,0.003363,NaN,0.023572
20160129,0.007451,-0.011741,0.008906,-0.123049,NaN,0.016452
20160429,0.000934,0.006463,0.001271,-0.014195,NaN,-0.000176
20160729,0.020894,0.021145,0.019743,-0.011719,NaN,0.020802
20161031,0.011858,0.016069,0.015851,0.043448,NaN,0.010548
20170126,-0.019690,-0.020954,-0.020338,-0.060613,NaN,-0.023250
20170428,-0.002897,-0.002623,-0.003590,-0.024449,NaN,-0.007383
